### Naive Softmax损失函数

In [1]:
def naiveSoftmaxLossAndGradient(
    centerWordVec, # 中心词的向量表示
    outsideWordIdx, # 正确的外部词（上下文词）在向量集中的索引
    outsideVectors, # 外部词的向量集合
    dataset 
):
    # 通过矩阵乘法计算每个外部词向量与中心词向量的点积，得到一个分数向量
    # 这个分数表示每个外部词与中心词的相似度。
    scores=np.matmul(outsideVectors, centerWordVec)
    # 应用softmax函数，将分数转换为概率分布
    # 对于每个外部词，其输出表示在给定中心词的情况下，该外部词被正确预测的概率。
    probs=softmax(scores)
    
    
    # 损失是正确外部词的概率的负对数
    # softmax回归中的典型损失函数，用于优化模型以正确预测正确的外部词。
    loss=-np.log(probs[outsideWordIdx]) # scalar
    
    # 复制概率分布。
    dscores=probs.copy
    # 对于正确的外部词，将其对应的概率减去1，得到梯度
    dscores[outsideWordIdx]=dscores[outsideWordIdx]-1
    # 中心词向量的梯度
    gradCenterVec=np.matmul(outsideVectors, dscores) # J关于vc的偏导公式
    # 外部词向量的梯度
    gradOutsideVecs=np.outer(dscores, centerWordVec) # J关于u的偏导公式
    
    return loss, gradCenterVec, gradOutsideVecs
# loss：损失值
# gradCenterVec：中心词向量的梯度
# gradOutsideVecs：外部词向量的梯度

### 负采样损失函数

In [2]:
def negSamplingLossAndGradient(
    centerWordVec, # 中心词的向量表示
    outsideWordIdx,
    outsideVectors,
    dataset,
    K=10
):
    # 在负采样框架下计算词嵌入模型的损失函数和梯度
  
    negSampleWordIndices = getNegativeSamples(outsideWordIdx, dataset, K)
    indices = [outsideWordIdx] + negSampleWordIndices

    gradCenterVec =np.zeros(centerWordVec.shape)  # (embedding_size,1)
    gradOutsideVecs = np.zeros(outsideVectors.shape)  # (vocab_size, embedding_size)
    loss = 0.0

    u_o = outsideVectors[outsideWordIdx]  # size=(embedding_size,1)
    z = sigmoid(np.dot(u_o, centerWordVec))  # size=(1, )
    loss -= np.log(z) # 损失函数的第一部分
    gradCenterVec += u_o * (z - 1)   # J关于vc的偏导数的第一部分
    gradOutsideVecs[outsideWordIdx] = centerWordVec * (z - 1)  # J关于u_o的偏导数计算

    for i in range(K):
        neg_id = indices[1 + i]
        u_k = outsideVectors[neg_id]
        z = sigmoid(-np.dot(u_k, centerWordVec))
        loss -= np.log(z)
        gradCenterVec += u_k * (1-z)
        gradOutsideVecs[neg_id] += centerWordVec * (1 - z)


    return loss, gradCenterVec, gradOutsideVecs
